# !pip install --upgrade -r ./config/requirements.txt
# !python3 -m pip install milvus milvus[client] pymilvus[model]
# Run `python3` in your terminal to operate in the Python interactive mode.


In [1]:

import sys
from os import environ
sys.path.append("./config/utils")




import numpy as np
import pandas as pd

from langchain_openai import OpenAIEmbeddings


from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import Milvus

from pymilvus import (MilvusClient,
                      CollectionSchema,
                      FieldSchema,
                      DataType,
                      utility,
                      db,
                      connections)



from MilvusControll import MilvusControll
from DocumentsWorkers import DocumentsWorkers


from Terminal import terminal_write
from constants import OPENAI_API_KEY
from constants import persist_directory

environ['OPENAI_API_KEY']=OPENAI_API_KEY
embeddings = OpenAIEmbeddings(model='text-embedding-3-large')


# Сортируем по желанию на документы из всех .txt в указаной директории 

In [2]:
folder_name='docs'

In [3]:
documents=list(filter(lambda document : '.txt' in document.__dict__['metadata']['source'], DocumentsWorkers.read_folder(folder_path=folder_name)))


In [4]:

documents=DocumentsWorkers.recursive_splitter.split_documents(documents=documents)
documents=DocumentsWorkers.documents_filter(documents=documents)
vectors=DocumentsWorkers.get_embedding_vectors(documents)
data=DocumentsWorkers.documents_to_dict_data(documents=documents, vectors=vectors)



KeyboardInterrupt



In [36]:
milvus=MilvusControll(collection_name='docs')

In [42]:
milvus.create_collection()

In [43]:
milvus.client.insert(collection_name='docs',
                     data=data)
    

{'insert_count': 188,
 'ids': ['2dc49aef-d63f-4f3b-b7a5-8efd33ec8e06', 'a5986a95-51e9-4a2a-8a73-7f16bd052c2d', '77d20b71-b819-4f77-9679-aa5fad462e94', '13b90263-606e-4072-9799-e464f75d7a0b', '2c5f1414-e9bd-40dc-b556-ebf1a5251f6a', 'ad779b0f-11e0-4fb5-b072-511d488491cc', '806b0695-6b01-475f-9dff-b46f3243c9b4', '91a6a248-c83e-49fa-975b-94dd8109023d', 'f91c1b22-dfbd-40db-83d5-9a02f4658b23', '7a9bb58c-4aab-450e-b843-75ec87b92c76', '933ca2a9-d61b-4d34-aaf7-2b0e4c9336d9', '0b3de069-d006-438a-b371-106ecc746769', 'ba5dba62-1a68-45d2-93be-5b1059f9c313', 'a556f17b-b843-4087-9cd6-a6ae3aee0808', '254623b2-7770-46f8-9208-e1831cc3f07f', 'd08929af-a353-4b01-9d4d-5bc41f26b775', 'b029dce7-7818-4eb0-814e-cb6f536f7022', '8d567554-cf2d-4ebb-995b-0d1b92a51ff0', '603ddda0-73fb-4df1-ad08-5dad5a6810e2', 'a4f36f63-f351-4d38-9787-8c4c5af9bc39', '6d3dff65-c838-43eb-bc57-0cdb4a0b4116', '79deab1d-0125-4713-8795-945bf3c1e168', '321975f2-ccad-493a-9ef4-7bb7c2327fab', 'a08f394d-36eb-4687-a7ea-d9c4cc35201b', '2dae8444

In [48]:
context=milvus.similarity_search(text='what 132',limit=5)

In [57]:
[index if 'School 125' in context[index]['text'] else None for index in range(len(context))]

[None, 1, 2, None, None]

In [ ]:
milvus.client.get()

['Ренар Лариса\nКруг женской силы. Энергии стихий и тайны обольщения',
 'Эта книга посвящается моим учителям — Светлане Дмитриевне Токтон, хранительнице древних знаний исчезнувшего континента My, научившей меня большинству женских практик; Алисии, передавшей мне северо-американскую традицию; Виргинии, открывшей мне тайны суфийской традиции и многим другим, кто посвятил свою жизнь раскрытию женской силы и поиску бесценных знаний. Эта книга посвящается всем ученицам и преподавателям «Академии частной жизни», и я благодарна миру за каждую встречу. Сколько волшебных и захватывающих часов мы провели вместе. Мы учились и менялись вместе и до сих пор многие с нами.',
 'Эта книга посвящается преподавателям петербургской «Академии частной жизни» — Ларисе Соколовой, Ольге Михайловой, Наталье Максимовой, а также основателям и преподавателям московской «Школы женских искусств» — Жене и Олегу Фроловым, Марианне, Ольге и др., которые каждый день делают этот мир лучше и добрее и наполняют его любовью

In [74]:
vectorstore=Chroma.from_documents(documents=documents[-5:], 
                      embedding=OpenAIEmbeddings(api_key=OPENAI_API_KEY),
                     persist_directory=persist_directory)

In [76]:
vectorstore.as_retriever(search_kwargs=dict(k=5))

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7cbfd6f1a190>, search_kwargs={'k': 5})

In [77]:
vectorstore.similarity_search('what 132')

[Document(metadata={'source': 'docs/1.txt'}, page_content='Увы, дорогие мои читательницы, я не могу все время быть рядом с вами, разоблачая лживые оправдания и ограждая вас от недостойных мужчин. Зато я могу нарисовать вам картину того, чего точно не случится в отношениях с мужчиной, который вас любит и ценит. Вы никогда не будете сидеть, отчаянно уставившись на телефон и усилием воли заставляя его зазвонить. Вы не станете портить себе встречу с подругами, каждые пятнадцать секунд проверяя входящие SMS. Вы не будете корить себя за то, что все же позвонили ему, хотя этого делать не следовало. А вот что выувидите– так это такое хорошее отношение к себе, что вам не понадобятся все эти телефонные ужимки и прыжки. Вы будете купаться в море нежности и внимания.Грег'),
 Document(metadata={'source': 'docs/1.txt'}, page_content='[Картинка: i_005.png] \xa0ВСЕ ТАК ПРОСТО'),
 Document(metadata={'source': 'docs/1.txt'}, page_content='А сейчас я выведу возмутительное, радикальное и суровое правило о

In [6]:
vectorstore = Milvus(collection_name="docs", embedding_function=DocumentsWorkers.embeddings)

In [7]:


vectorstore.as_retriever(search_kwargs={'k':5}).invoke("what 132")


[Document(metadata={'id': 'f33ae36f-f785-4ccc-96d3-47c534eb1d93', 'source': 'docs/school_informations.txt'}, page_content='School 130|||Armenia, Yerevan, Hakob Hakobyan St. Lane, 3 Building (Arabkir adm. district)\nSchool 131|||Armenia, Yerevan, Arzumanyan St., 22 Building (Ajapnyak adm. district)\nSchool 132|||Armenia, Yerevan, Aram Khachatryan St., 28 Building (Arabkir adm. district)\nSchool 133|||Armenia, Yerevan, Nor Nork 2nd Block, Moldovakan St., 35 Building (Nor Nork adm. district)\nSchool 134|||Armenia, Yerevan, Nor Nork 1st Block, Nanseni St., 14 Building (Nor Nork adm. district)'),
 Document(metadata={'id': 'a29a2ed9-49d3-4dce-9ece-d17b9d052ceb', 'source': 'docs/school_informations.txt'}, page_content='School 134|||Armenia, Yerevan, Nor Nork 1st Block, Nanseni St., 14 Building (Nor Nork adm. district)\nSchool 135|||Armenia, Yerevan, Nor Nork 2nd Block, Nelson Stepanyan St., 5 Building (Nor Nork adm. district)\nSchool 136|||Armenia, Yerevan, Lepsiusi St., 6 Building (Kanaker-Z